# Рейтинг-система для спортивного "Что? Где? Когда?"

In [1]:
import random
import pickle
from collections import defaultdict
from itertools import chain

import pandas as pd
import numpy as np

from sklearn.linear_model import LogisticRegression, Ridge
from scipy.stats import spearmanr, kendalltau
from scipy.sparse import csr_matrix

## 1. Подготовка данных

### Описание полей итогового датасета

- $\textbf{cid}$ (contest id) - id турнира
- $\textbf{tid}$ (team id) - id команды
- $\textbf{pid}$ (player id) - id игрока
- $\textbf{bit}$ - результат ответа на вопрос
- $\textbf{mask}$ - результат ответов на вопросы
- $\textbf{team}$ - список id игроков
- $\textbf{position}$ - место по итогам турнира

### Загрузка

In [2]:
PATH_PLAYERS = "data/players.pkl"
PATH_RESULTS = "data/results.pkl"
PATH_CONTESTS = "data/tournaments.pkl"

In [3]:
def get_data(path):
    with open(path, 'rb') as fi:
        return pickle.load(fi)

In [4]:
PLAYERS_DICT = get_data(PATH_PLAYERS)
RESULTS_DICT = get_data(PATH_RESULTS)
CONTESTS_DICT = get_data(PATH_CONTESTS)

In [5]:
TEAM_ID_TO_NAME = {}

for k, v in RESULTS_DICT.items():
    for record in v:
        team = record['team']
        if team['id'] not in TEAM_ID_TO_NAME:
            TEAM_ID_TO_NAME[team['id']] = team['name']

In [6]:
df_raw = pd.DataFrame([v for _, v in CONTESTS_DICT.items()])
df_raw.tail(3)

,id,name,dateStart,dateEnd,type,season,orgcommittee,synchData,questionQty
5525,6483,Онлайн: 19:00 (а)Синхрон-lite. Лига старта. Эп...,2020-05-08T19:00:00+03:00,2020-05-08T21:30:00+03:00,"{'id': 2, 'name': 'Обычный'}",/seasons/53,"[{'id': 7533, 'name': 'Денис', 'patronymic': '...",None,"{'1': 12, '2': 12, '3': 12}"
5526,6484,"Онлайн: 22:00 Не числом, а умением - 2 (NEW!)",2020-05-04T22:00:00+03:00,2020-05-04T23:40:00+03:00,"{'id': 2, 'name': 'Обычный'}",/seasons/53,"[{'id': 7533, 'name': 'Денис', 'patronymic': '...",None,"{'1': 12, '2': 12}"
5527,6485,"Онлайн: 19:00 Не числом, а умением",2020-05-06T19:00:00+03:00,2020-05-06T20:45:00+03:00,"{'id': 2, 'name': 'Обычный'}",/seasons/53,"[{'id': 7533, 'name': 'Денис', 'patronymic': '...",None,"{'1': 12, '2': 12}"


In [7]:
mask_train= (df_raw['dateStart'] > "2019") & (df_raw['dateStart'] < "2020")
mask_test = df_raw['dateStart'] > "2020"

cid_collection_train = df_raw[mask_train]['id'].to_list()
cid_collection_test = df_raw[mask_test]['id'].to_list()

### Очистка

In [8]:
def extract_info(cid_collection):
    data = []
    columns = ["cid", "tid", "position", "mask", "team"]
    for cid in cid_collection:
        for team in RESULTS_DICT[cid]:
            if any(key not in team for key in ["mask", "teamMembers", "position", "team"]):
                continue
            if team["mask"] is None or not team["mask"]:
                continue
            pid_collection = [member["player"]["id"] for member in team["teamMembers"]]
            if pid_collection:
                data.append((cid, team["team"]["id"], team["position"], team["mask"], pid_collection))
    return pd.DataFrame(data, columns=columns)

In [9]:
df_train = extract_info(cid_collection_train)
df_test = extract_info(cid_collection_test)

df_train.head(3)

,cid,tid,position,mask,team
0,4772,45556,1.0,111111111011111110111111111100010010,"[6212, 18332, 18036, 22799, 15456, 26089]"
1,4772,1030,5.5,111111111011110100101111011001011010,"[1585, 40840, 1584, 10998, 16206]"
2,4772,4252,5.5,111111111011110101101111001011110000,"[23513, 18168, 21060, 35850, 31332, 10187]"


In [10]:
print('Размер обучающей выборки:', df_train.shape[0])
print('Размер тестовой выборки:', df_test.shape[0])

Размер обучающей выборки: 86395
Размер тестовой выборки: 22366


In [11]:
def clear_competitions(df):
    """Удаляет турниры, где количество вопросов разнится."""
    # сколько различных по длине последовательностей вопросов на одном турнире
    func = lambda results: len({len(r) for r in results})
    # id турниров, где было разное количество вопросов
    cid_count_dict = df.groupby('cid')['mask'].apply(list).apply(func).to_dict()
    cid_collection = [cid for cid, count in cid_count_dict.items() if count > 1]
    # исключим эти турниры
    return df[~df['cid'].isin(cid_collection)]
    

def clear_questions(df):
    """Удаляет вопросы, где в маске не 0 или не 1."""
    f_indices = lambda results: {i for result in results for i, q in enumerate(result) if q not in ["0", "1"]}
    cid_indices_dict = df.groupby('cid')['mask'].apply(list).apply(f_indices).to_dict()
    f_clean = lambda row: ''.join([q for i, q in enumerate(row[1]) if i not in cid_indices_dict[row[0]]])
    return df[['cid', 'mask']].apply(f_clean, axis=1)

In [12]:
df_train = clear_competitions(df_train)
df_train['mask'] = clear_questions(df_train)

print('Размер обучающей выборки:', df_train.shape[0])

Размер обучающей выборки: 78410


In [13]:
df_train["mask"] = df_train["mask"].apply(lambda mask: [int(bit) for bit in mask])
df_train.head(3)

,cid,tid,position,mask,team
0,4772,45556,1.0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, ...","[6212, 18332, 18036, 22799, 15456, 26089]"
1,4772,1030,5.5,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, ...","[1585, 40840, 1584, 10998, 16206]"
2,4772,4252,5.5,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, ...","[23513, 18168, 21060, 35850, 31332, 10187]"


### Формирование итогового датасета и вспомогательных данных

In [14]:
cid_collection = df_train['cid'].unique()

# "qid" -> (cid, index in mask)
qid_to_cid = {}
# {"cid": {"tid": [list of pids]}}
cid_tid_to_pids = {}
# {"cid": {"tid": [list of bits]}}
cid_tid_to_bits = {}

df_collection = []
counter = 1

for cid in cid_collection:

    df = df_train[df_train['cid'] == cid].drop(columns=["position"]).copy()
    cid_tid_to_pids[cid] = dict(zip(df["tid"], df["team"]))
    cid_tid_to_bits[cid] = dict(zip(df["tid"], df["mask"]))
        
    df = df.explode("team")
        
    ids = df[["cid", "tid", "team"]].to_numpy()
    columns_ids = ["cid", "tid", "pid"]    
    mask = np.array(df["mask"].apply(lambda mask: [int(bit) for bit in mask]).to_list())
    n_questions = mask.shape[1]
    columns_mask = [i for i in range(counter, counter + n_questions)]
    counter += n_questions
    
    for i, qid in enumerate(columns_mask):
        qid_to_cid[qid] = (cid, i)
            
    df = pd.DataFrame(np.hstack([ids, mask]), columns=columns_ids + columns_mask)
    
    # competition_id + team_id + player_id + question_id + bit
    data_melted = df.melt(id_vars=columns_ids, var_name="qid", value_name="bit").to_numpy()
    df_collection.append(data_melted)

df_train_long = pd.DataFrame(np.vstack(df_collection), columns=['cid', 'tid', 'pid', 'qid', 'bit'])
df_train_long.sort_values(by=['pid', 'cid', 'qid'], inplace=True)

In [15]:
df_train_long.head(5)

,cid,tid,pid,qid,bit
46288,4973,51584,15,37,1
51560,4973,51584,15,38,1
56832,4973,51584,15,39,1
62104,4973,51584,15,40,0
67376,4973,51584,15,41,0


In [16]:
# {"pid": [list of cids]}
pid_to_cids = df_train_long.groupby('pid')['cid'].apply(set).apply(sorted).to_dict()

# {"pid": {"cid": [list of pids]}}
PID_CID_TO_PIDS_BITS = defaultdict(dict)

for cid, tids in cid_tid_to_pids.items():
    for tid, pids in tids.items():
        for pid in pids:
            if cid in PID_CID_TO_PIDS_BITS[pid]:
                continue
            bits = cid_tid_to_bits[cid][tid]
            PID_CID_TO_PIDS_BITS[pid][cid] = {'pids': pids, 'bits': bits}
            
pid_cid_to_indices = df_train_long.groupby(['pid', 'cid']).indices

pid_collection = df_train_long['pid'].unique()
n_players = pid_collection.shape[0]

print("Количество игроков, принимавших участие в турнирах 2019 года:", n_players)

Количество игроков, принимавших участие в турнирах 2019 года: 56938


### Обучающая выборка

In [17]:
df_ohe = pd.get_dummies(df_train_long, columns=["pid", "qid"], sparse=True)
df_ohe

,cid,tid,bit,pid_15,pid_16,pid_23,pid_31,pid_35,pid_38,pid_47,...,qid_30881,qid_30882,qid_30883,qid_30884,qid_30885,qid_30886,qid_30887,qid_30888,qid_30889,qid_30890
46288,4973,51584,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
51560,4973,51584,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
56832,4973,51584,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
62104,4973,51584,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
67376,4973,51584,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12850912,5828,76113,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12851703,5828,76113,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12852494,5828,76113,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12853285,5828,76113,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
X_train = df_ohe.drop(columns=["cid", "tid", "bit"]).sparse.to_coo()

## 2. Baseline-модель (логистическая регрессия)

- веса модели будут отражать мастерство игрока и сложность вопроса (в соответствии со столбцами X_train)
- примем за 0 мастерство начинающего или игрока, не участвовавшего в турнирах 2019 года

In [19]:
DEFAULT_SKILL = 0
COEFF_REG_LOGREG = 1
MAX_ITER = 1000

In [20]:
y_train = df_ohe["bit"].astype('int')
model_logreg = LogisticRegression(C=COEFF_REG_LOGREG, penalty='l2', max_iter=MAX_ITER).fit(X_train, y_train)

In [21]:
def get_rating(model):
    skill_collection = model.coef_[:n_players].flatten()
    skill_collection -= skill_collection.min()
    rating = []
    columns = ['pid', 'name', 'surname', 'skill']
    for pid, skill in zip(pid_collection, skill_collection):
        p = PLAYERS_DICT[pid]
        rating.append((pid, p['name'], p['surname'], round(skill, 4)))
    df_rating = pd.DataFrame(rating, columns=columns)
    df_rating['n_questions'] = df_rating['pid'].map(df_train_long.groupby('pid')['qid'].nunique().to_dict())
    df_rating['n_competitions'] = df_rating['pid'].map(df_train_long.groupby('pid')['cid'].nunique().to_dict())
    df_rating.sort_values(by='skill', inplace=True, ascending=False)
    df_rating.index = range(1, df_rating.shape[0] + 1)
    most_common_team_dict = df_train.explode('team').groupby(
        ['team', 'tid'], as_index=False).count().sort_values(
        by=['team', 'cid'], ascending=[True, False]).groupby(
        'team').head(1).set_index('team')['tid'].map(TEAM_ID_TO_NAME).to_dict()
    df_rating['most_common_team'] = df_rating['pid'].map(most_common_team_dict)
    return df_rating

In [22]:
df_rating_logreg = get_rating(model_logreg)
rating_logreg_dict = df_rating_logreg.set_index('pid')['skill'].to_dict()

In [23]:
df_rating_logreg.head(30)

,pid,name,surname,skill,n_questions,n_competitions,most_common_team
1,27403,Максим,Руссо,9.8495,2176,55,Борский корабел
2,4270,Александра,Брутер,9.7233,2690,67,Борский корабел
3,28751,Иван,Семушин,9.6750,3771,95,Борский корабел
4,27822,Михаил,Савченков,9.5860,3211,79,Борский корабел
5,30270,Сергей,Спешков,9.5088,3733,87,Борский корабел
6,30152,Артём,Сорожкин,9.5042,4845,124,Борский корабел
7,20691,Станислав,Мереминский,9.3886,1581,37,Ксеп
8,18036,Михаил,Левандовский,9.3286,1454,33,Рабочее название
9,26089,Ирина,Прокофьева,9.2851,1061,26,Рабочее название
10,22799,Сергей,Николенко,9.2711,2215,50,Рабочее название


- Согласуется с реальным рейтингом игроков
- Присутствуют топовые команды - Борский корабел, Рабочее название, Команда Губанова

## 3. Предсказание результатов на тестовой выборке

- Результат команды будем считать как среднее ее трех лучших игроков 

In [24]:
random.seed(0)


def calculate_team_score(team, rating_dict, n=3):
    rating_collection = []
    for pid in team:
        if pid in rating_dict:
            rating_collection.append(rating_dict[pid])
        else:
            rating_collection.append(DEFAULT_SKILL)
    rating_collection.sort(reverse=True)
    return np.array(rating_collection[:n]).mean()


def get_score(df_test, rating_dict, n):

    score_spearman = []
    score_kendall = []

    for cid in df_test['cid'].unique():
        df = df_test[df_test['cid'] == cid]
        y_true = df['position'].to_list()
        
        if len(y_true) < 2:
            continue
        
        predicted_result = list(zip(df['tid'], df['team'].apply(calculate_team_score, args=(rating_dict, n))))
        random.shuffle(predicted_result)
        
        y_pred = [tid for tid, score in sorted(predicted_result, key=lambda x: x[1], reverse=True)]
        y_pred = df['tid'].map(dict(zip(y_pred, range(1, len(y_pred) + 1)))).to_list()

        score_spearman.append(spearmanr(y_pred, y_true)[0])
        score_kendall.append(kendalltau(y_pred, y_true)[0])

    return np.array(score_spearman).mean(), np.array(score_kendall).mean()

In [25]:
for n in range(1, 7):
    s, k = get_score(df_test, rating_logreg_dict, n)
    print(f'Количество игроков: {n}, корреляция Спирмена: {s:.4f}, корреляция Кендалла: {k:.4f}')

Количество игроков: 1, корреляция Спирмена: 0.7690, корреляция Кендалла: 0.6088
Количество игроков: 2, корреляция Спирмена: 0.7932, корреляция Кендалла: 0.6339
Количество игроков: 3, корреляция Спирмена: 0.7930, корреляция Кендалла: 0.6353
Количество игроков: 4, корреляция Спирмена: 0.7855, корреляция Кендалла: 0.6281
Количество игроков: 5, корреляция Спирмена: 0.7698, корреляция Кендалла: 0.6133
Количество игроков: 6, корреляция Спирмена: 0.7462, корреляция Кендалла: 0.5885


- Оценка силы команды расчитывается как среднее из мастерства игроков команды
- Наибольшие значение метрики достигается, если из команды взять двух или трех ее лучших игроков

## 4. Построение рейтинга при помощи ЕМ-алгоритма

- Введем скрытую переменную - ответил ли игрок на конкретный вопрос турнира
- Целевая переменная логистической регресии - вероятность того, ответил ли игрок на вопрос
- Модель будем обучать при помощи линейной регрессии
- Сделаем предположение, что команда ответила на вопрос, если хотя бы один из игроков ответил на вопрос

### Необходимые функции

In [26]:
def get_z_updated(cid, pid, z_collection):
    
    pids_bits = PID_CID_TO_PIDS_BITS[pid][cid]
    pids = pids_bits['pids']

    z_team = []
    for pid in pids:
        indices = pid_cid_to_indices[(pid, cid)]
        z = z_collection[indices].flatten()
        z_team.append(z)
    z_team = np.vstack(z_team)

    # вероятность команды ответить на вопрос (хотя бы один из игроков ответил)
    p_team = 1 - np.prod(1 - z_team, axis=0)
    # условная вероятность игрока ответить на вопрос
    z_team /= p_team
    # наложим маску (обнулим вероятности у вопросов, где не было ответа команды)
    z_team *= np.array(pids_bits['bits'])
                     
    return dict(zip(pids, z_team))


def get_z_full(pid, z_updated_collection):
    z_full = []
    for cid in sorted(z_updated_collection[pid].keys()):
        z_full.append(z_updated_collection[pid][cid])
    return np.hstack(z_full)


def logit(x, eps=10 ** -7):
    x = np.clip(x, eps, 1 - eps)
    return -np.log((1 - x) / x)


def siqmoid(x):
    return 1 / (1 + np.exp(-x))

In [27]:
def m_step(X_train, z, alpha=5):
    y = logit(z)
    model = Ridge(alpha=alpha, fit_intercept=True).fit(X_train, y)
    return model


def e_step(model, X_train, shift_collection):
    
    y_pred = X_train @ model.coef_.reshape(-1, 1) + model.intercept_
    z_collection = siqmoid(y_pred + shift_collection)
    
    # {"pid": {"cid": [z_updated]}}
    z_updated_collection = defaultdict(dict)
    for pid in pid_collection:
        for cid in pid_to_cids[pid]:
            if pid in z_updated_collection and cid in z_updated_collection[pid]:
                continue
            team_result = get_z_updated(cid, pid, z_collection)
            for pid_from_team, z_updated in team_result.items():
                z_updated_collection[pid_from_team][cid] = z_updated
                
    z_long = []
    finished = set()
    for pid in df_train_long['pid']:
        if pid in finished:
            continue
        finished.add(pid)
        z_full = get_z_full(pid, z_updated_collection)
        z_long.append(z_full)
        
    return np.hstack(z_long)


def calculate_shift(pi):
    # повопросные результаты для всех турниров
    mask_all_results = np.fromiter(chain.from_iterable(df_train['mask'].to_list()), dtype='int')
    # количество примеров, где команда ответила на вопрос (неизвестного состояние)
    n_unk = (mask_all_results == 1).sum()
    # количество примеров, где команда не ответила на вопрос
    n_neg = (mask_all_results == 0).sum()
    # вероятность игрока ответить: часть неизвестного (команда ответила)
    pi_pos = (pi * n_unk) / (n_unk + n_neg)
    # вероятность игрока не ответить: неотвеченные (команда не ответила) + часть неизвестного (команда ответила)
    pi_neg = (n_neg + (1 - pi) * n_unk) / (n_unk + n_neg)
    return np.log(pi_neg / pi_pos)


def calculate_team_prior():
    """Априорная вероятность того, что команда ответит на вопрос"""
    return df_train_long['bit'].mean()

In [28]:
def calculate_shift(mask, pi):
    """Считает сдвиг для одного игрока"""
    # повопросные результаты всех турниров
    mask = np.array(mask)
    # количество примеров, где команда игрока ответила на вопрос (неизвестного состояние)
    n_unk = (mask == 1).sum()
    # количество примеров, где команда игрока не ответила на вопрос
    n_neg = (mask == 0).sum()
    # вероятность игрока ответить: часть неизвестного (команда ответила)
    pi_pos = (pi * n_unk) / (n_unk + n_neg)
    # вероятность игрока не ответить: неотвеченные (команда не ответила) + часть неизвестного (команда ответила)
    pi_neg = (n_neg + (1 - pi) * n_unk) / (n_unk + n_neg)
    return np.log(pi_neg / pi_pos)


def calculate_shift_collection(pi):
    pid_to_shift = df_train_long.groupby('pid')['bit'].apply(list).apply(calculate_shift, args=(pi,))
    return df_train_long.pid.map(pid_to_shift.to_dict()).to_numpy().reshape(-1, 1)

### Обучение модели

- E-шаг: инициализируем вектор z априорными вероятностями
- M-шаг: обучаем логистическую регрессию при помощи линейной с L2 регуляризацией, где целевая переменная равна logit(z). Получаем параметры модели w
- E-шаг: получаем новый вектор вероятностей z на основе параметров модели и наших предположений

In [45]:
N_ITERS = 20
ALPHA = 5 
P_PRIOR = 1 - (1 - calculate_team_prior()) ** (1 / 6)
N_BEST = 3

In [46]:
shift_collection = calculate_shift_collection(P_PRIOR)
z = df_ohe["bit"].copy().to_numpy().astype('float32') * P_PRIOR

best_model_linreg = None
best_score_linreg = 0

for i in range(N_ITERS):
    
    # M-шаг
    model_linreg = m_step(X_train, z, alpha=ALPHA)
    
    # подсчет метрик
    df_rating_linreg = get_rating(model_linreg)
    rating_linreg_dict = df_rating_linreg.set_index('pid')['skill'].to_dict()  
    s, k = get_score(df_test, rating_linreg_dict, n=N_BEST)
    
    if s > best_score_linreg:
        best_score_linreg = s
        best_model_linreg = model_linreg
    
    step = ' ' + str(i + 1) if i + 1 < 10 else i + 1
    print(f'Шаг обучения: {step}, Корреляция Спирмена: {s:.4f}, корреляция Кендалла: {k:.4f}')
    
    if i + 1 == N_ITERS:
        break
    
    # E-шаг
    z = e_step(model_linreg, X_train, shift_collection)

Шаг обучения:  1, Корреляция Спирмена: 0.7902, корреляция Кендалла: 0.6317
Шаг обучения:  2, Корреляция Спирмена: 0.7961, корреляция Кендалла: 0.6362
Шаг обучения:  3, Корреляция Спирмена: 0.8011, корреляция Кендалла: 0.6422
Шаг обучения:  4, Корреляция Спирмена: 0.8007, корреляция Кендалла: 0.6415
Шаг обучения:  5, Корреляция Спирмена: 0.8011, корреляция Кендалла: 0.6420
Шаг обучения:  6, Корреляция Спирмена: 0.8014, корреляция Кендалла: 0.6424
Шаг обучения:  7, Корреляция Спирмена: 0.8032, корреляция Кендалла: 0.6441
Шаг обучения:  8, Корреляция Спирмена: 0.8008, корреляция Кендалла: 0.6415
Шаг обучения:  9, Корреляция Спирмена: 0.8010, корреляция Кендалла: 0.6421
Шаг обучения: 10, Корреляция Спирмена: 0.8011, корреляция Кендалла: 0.6421
Шаг обучения: 11, Корреляция Спирмена: 0.7997, корреляция Кендалла: 0.6411
Шаг обучения: 12, Корреляция Спирмена: 0.8006, корреляция Кендалла: 0.6419
Шаг обучения: 13, Корреляция Спирмена: 0.8008, корреляция Кендалла: 0.6419
Шаг обучения: 14, Корреля

- Значение метрик выше, чем у логистической регрессии (корреляция Спирмена 0.803 против 0.793)

## 5. Построение рейтинга турниров

- Возьмем результаты бейзлайна и EM-алгоритма
- Рейтинг турнира - среднее из сложностей вопросов
- Посмотрим на десятку самых сложных и самых легких турниров

In [47]:
def get_competition_rating(model):
    q_level = -model.coef_.flatten()[n_players:]
    q_level -= q_level.min()
    qid_to_level_dict = dict(zip(range(1, q_level.shape[0] + 1), q_level))
    df_comps = df_train_long[['cid', 'qid']].copy()
    df_comps['q_level'] = df_comps['qid'].map(qid_to_level_dict)
    df_comps = df_comps.groupby('cid', as_index=False)['q_level'].mean()
    df_comps.columns = ['cid', 'difficulty']
    df_comps['title'] = df_comps['cid'].apply(lambda cid: CONTESTS_DICT[cid]['name'])
    df_comps.sort_values(by='difficulty', ascending=False, inplace=True)
    return df_comps

In [48]:
# logreg baseline
df_comp_rating_logreg = get_competition_rating(model_logreg)
# linreg EM-algorithm
df_comp_rating_linreg = get_competition_rating(best_model_linreg)

### Верхняя часть таблицы

In [49]:
df_comp_rating_logreg.head(10)

,cid,difficulty,title
654,6149,10.569889,Чемпионат Санкт-Петербурга. Первая лига
535,5928,8.441399,Угрюмый Ёрш
366,5684,8.236049,Синхрон высшей лиги Москвы
43,5159,8.213027,Первенство правого полушария
630,6101,8.124577,Воображаемый музей
281,5587,7.939540,Записки охотника
13,5025,7.867113,Кубок городов
374,5693,7.833246,Знание – Сила VI
26,5083,7.827598,Ускользающая сова
49,5186,7.778685,VERSUS: Коробейников vs. Матвеев


In [50]:
df_comp_rating_linreg.head(10)

,cid,difficulty,title
654,6149,37.369642,Чемпионат Санкт-Петербурга. Первая лига
535,5928,35.222835,Угрюмый Ёрш
366,5684,34.313875,Синхрон высшей лиги Москвы
630,6101,34.134543,Воображаемый музей
43,5159,34.021822,Первенство правого полушария
550,5946,33.861286,Чемпионат Мира. Этап 3. Группа В
178,5465,33.859397,Чемпионат России
546,5942,33.827958,Чемпионат Мира. Этап 2. Группа В
374,5693,33.773480,Знание – Сила VI
281,5587,33.624151,Записки охотника


Содержание таблиц соответствует логике - в топе такие турниры как:
- чемпионат мира
- чемпионат России
- чемпионаты Москвы, Санкт-Петербурга

### Нижняя часть таблицы

In [51]:
df_comp_rating_logreg.tail(10)

,cid,difficulty,title
72,5313,4.196637,(а)Синхрон-lite. Лига старта. Эпизод VI
540,5936,4.193411,Школьная лига. I тур.
9,5011,4.143925,(а)Синхрон-lite. Лига старта. Эпизод IV
172,5457,4.141189,Студенческий чемпионат Калининградской области
10,5012,4.034409,Школьный Синхрон-lite. Выпуск 2.5
377,5698,3.953591,(а)Синхрон-lite. Лига старта. Эпизод VII
381,5702,3.939238,(а)Синхрон-lite. Лига старта. Эпизод IX
7,5009,3.909283,(а)Синхрон-lite. Лига старта. Эпизод III
154,5438,3.908646,Синхрон Лиги Разума
11,5013,3.707622,(а)Синхрон-lite. Лига старта. Эпизод V


In [52]:
df_comp_rating_linreg.tail(10)

,cid,difficulty,title
382,5704,22.285955,(а)Синхрон-lite. Лига старта. Эпизод X
72,5313,21.655429,(а)Синхрон-lite. Лига старта. Эпизод VI
463,5828,21.652271,Гарри Поттер и 3 по 12
377,5698,21.456873,(а)Синхрон-lite. Лига старта. Эпизод VII
172,5457,21.431641,Студенческий чемпионат Калининградской области
7,5009,21.109873,(а)Синхрон-lite. Лига старта. Эпизод III
381,5702,21.093459,(а)Синхрон-lite. Лига старта. Эпизод IX
592,6003,20.919788,Второй тематический турнир имени Джоуи Триббиани
11,5013,20.565750,(а)Синхрон-lite. Лига старта. Эпизод V
154,5438,18.201547,Синхрон Лиги Разума


В подвале таблицы превалируют:
- Турниры с меткой "lite" и "старт"
- Студенческие и школьные турниры

## 6. Рейтинг игроков на основе EM-алгоритма

- В топе присутствует много игроков с небольшим количеством сыгранных турниров
- После отфильтровки по количеству турниров (минимум 10) лидерство вернула команда Борский корабел, а топ пополнили игроки команд Рабочее название, Команда Губанова

In [53]:
df_rating_linreg = get_rating(best_model_linreg)

In [54]:
df_rating_linreg.head(20)

,pid,name,surname,skill,n_questions,n_competitions,most_common_team
1,22474,Илья,Немец,24.2850,75,2,Сумрачный германский
2,27403,Максим,Руссо,22.9386,2176,55,Борский корабел
3,4270,Александра,Брутер,22.1501,2690,67,Борский корабел
4,28751,Иван,Семушин,22.0705,3771,95,Борский корабел
5,30152,Артём,Сорожкин,21.7914,4845,124,Борский корабел
6,27822,Михаил,Савченков,21.7811,3211,79,Борский корабел
7,38175,Максим,Пилипенко,21.5197,36,1,Алиса в Закавказье
8,20691,Станислав,Мереминский,21.4052,1581,37,Ксеп
9,7008,Алексей,Гилёв,21.2860,4298,110,Команда Губанова
10,30270,Сергей,Спешков,21.2648,3733,87,Борский корабел


In [59]:
df_rating_linreg[df_rating_linreg['n_competitions'] >= 10].head(20)

,pid,name,surname,skill,n_questions,n_competitions,most_common_team
2,27403,Максим,Руссо,22.9386,2176,55,Борский корабел
3,4270,Александра,Брутер,22.1501,2690,67,Борский корабел
4,28751,Иван,Семушин,22.0705,3771,95,Борский корабел
5,30152,Артём,Сорожкин,21.7914,4845,124,Борский корабел
6,27822,Михаил,Савченков,21.7811,3211,79,Борский корабел
8,20691,Станислав,Мереминский,21.4052,1581,37,Ксеп
9,7008,Алексей,Гилёв,21.2860,4298,110,Команда Губанова
10,30270,Сергей,Спешков,21.2648,3733,87,Борский корабел
15,15727,Александр,Коробейников,20.9824,1401,29,Слон в удаве
16,18332,Александр,Либер,20.8553,3780,92,Рабочее название
